# <h1 align=center> **PROYECTO INDIVIDUAL Nº1** </h1>

PI Data Engineering: Jhoeliel Palma Salazar

In [1]:
import pandas as pd

### **Requerimientos del área de análisis de datos**

Definicion de plataformas en una lista que será usada en el bucle de carga

In [2]:
plataformas=['amazon_prime_titles-score.csv','disney_plus_titles-score.csv','hulu_titles-score.csv','netflix_titles-score.csv']
df_completo=pd.DataFrame()

**Proceso de transformaciones:**
Se hará una iteracion por cada plataforma de la lista. El nombre de la plataforma de la lista coincide con el nombre del archivo del dataset correspondiente, con la finalidad de simplificar el proceso

In [3]:
# ETL
for nombre in plataformas:
    df = pd.read_csv("Datasets/"+nombre, sep=",", encoding="UTF-8")
    # Campo id con la primera letra de la plataforma + show_id
    df["id"] = nombre[0] + df["show_id"] 
    # Moviendo columna a la posicion 0
    df_pop = df.pop('id') 
    df.insert(0,'id', df_pop)
    # Reeplazando valores nulos en 'rating' por G
    df.loc[df['rating'].isnull(), 'rating'] = "G"
    # Convirtiendo a tipo fecha
    df['date_added'] = pd.to_datetime(df['date_added'])
    # Cambiando formato de fecha a AAAA-mm-dd
    df['date_added'] = pd.to_datetime(df['date_added'], format="%YYYY-%mm-%dd")
    # Convirtiendo a minusculas todos los campos de texto
    df = df.apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)
    # Convirtiendo campo duration en dos campos: duration_int y duration_type
    df[['duration_int','duration_type']] = df['duration'].str.extract(r'(\d+)\s*(\w+)', expand=True).fillna(0)
    df['duration_int'] = df['duration_int'].astype(int)
    df['duration_type'].replace('seasons','season',inplace=True)
    # Moviendo columnas 
    df_pop = df.pop('duration_int') 
    df.insert(11,'duration_int', df_pop)
    df_pop = df.pop('duration_type') 
    df.insert(12,'duration_type', df_pop)
    # Eliminando 'duration'
    df = df.drop(columns=['duration'])
    #df['score'] = df['score'].astype(int)
    df_completo = pd.concat([df_completo, df], ignore_index=True)
    df_completo = df_completo.infer_objects()

In [11]:
df_completo.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration_int,duration_type,listed_in,description,score
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113,min,"comedy, drama",a small fishing village must procure a local d...,99
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110,min,"drama, international",a metro family decides to fight a cyber crimin...,37
2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74,min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20
3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69,min,documentary,"pink breaks the mold once again, bringing her ...",27
4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45,min,"drama, fantasy",teenage matt banting wants to work with a famo...,75


Una vez finalizado el ETL se procedera a escribir el artchivo .csv desde el dataframe que contiene el dataset completo y unificado

In [4]:
df_completo.to_csv("Datasets/completo_.csv", sep=",", encoding="UTF-8")

En esta parte probaremos las APIs de las consultas requeridas antes de ser subidas

1. Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma

In [5]:
# API 1

def get_word_count(plataforma:str,keyword:str): # 2 parametros de entrada: 2 strings
    df_completo = pd.read_csv('Datasets/completo.csv')
    df_filtrado = df_completo[df_completo['id'].str.contains(plataforma[0])]
    count = int(df_filtrado['title'].str.contains(keyword).sum())
    return plataforma,count # 2 valores de salida 1 string y 1 entero

print(get_word_count('netflix','love'))

('netflix', 196)


2. Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año

In [6]:
# API 2

def get_score_count(plataforma:str,score:int,year:int): # 3 parametros de entrada: 1 string y 2 enteros
    df_completo = pd.read_csv('Datasets/completo.csv')
    df_filtrado = df_completo[df_completo['id'].str.contains(plataforma[0])]
    df_filtrado = df_filtrado[df_filtrado['release_year'] == year]
    df_filtrado = df_filtrado[df_filtrado['type'] == 'movie']
    count=int(df_filtrado[df_filtrado['score'] > score].shape[0])
    return plataforma,count # 2 valores de salida 1 string y 1 entero

print(get_score_count('netflix',85,2010))

('netflix', 20)


3. La segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos.

In [7]:
# API 3

def get_second_score(plataforma:str): # 1 parametro de entrada: 1 string
    df_completo = pd.read_csv('Datasets/completo.csv')
    df_filtrado=df_completo[df_completo['id'].str.contains(plataforma[0])]
    df_filtrado = df_filtrado[df_filtrado['type'] == 'movie']
    df_filtrado = df_filtrado.sort_values(by=['score','title'], ascending=[False, True])
    df_filtrado= df_filtrado.reset_index(drop=True)
    title=df_filtrado['title'][1]
    score=int(df_filtrado['score'][1])
    return title,score # 2 valores de salida 1 string y 1 entero

print(get_second_score('amazon'))

('abilene town', 100)


4. Película que más duró según año, plataforma y tipo de duración

In [8]:
# API 4

def get_longest(plataforma:str,duracion:str,year:int): # 3 parametros de entrada: 2 strings y 1 entero
    #df_completo = pd.read_csv('https://raw.githubusercontent.com/Jhoeliel/PI-DataEngineering/main/completo.csv')
    df_completo = pd.read_csv('Datasets/completo.csv')
    df_filtrado=df_completo[df_completo['id'].str.contains(plataforma[0])]
    df_filtrado = df_filtrado[df_filtrado['release_year'] == year]
    df_filtrado = df_filtrado[df_filtrado['duration_type'].str.contains(duracion)]
    df_filtrado = df_filtrado.sort_values(by=['duration_int'], ascending=[False])
    df_filtrado= df_filtrado.reset_index(drop=True)
    title=df_filtrado['title'][0]
    duracion_int=int(df_filtrado['duration_int'][0])
    duracion_type=df_filtrado['duration_type'][0]
    return title,duracion_int,duracion_type # 3 valores de salida 1 string, 1 entero y 1 string

print(get_longest('netflix','min',2016))

('sairat', 173, 'min')


5. Cantidad de series y películas por rating

In [10]:
# API 5

def get_rating_count(rating:str): # 1 parametro de entrada: 1 string
    #df_completo = pd.read_csv('https://raw.githubusercontent.com/Jhoeliel/PI-DataEngineering/main/completo.csv')
    df_completo = pd.read_csv('Datasets/completo.csv')
    df_filtrado = df_completo[df_completo['rating']==rating]
    cantidad=int(df_filtrado.shape[0])
    return rating,cantidad # 2 valores de salida 1 string y 1 entero

print(get_rating_count("18+"))

('18+', 1243)
